In [ ]:
import numpy as np
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# for classifier
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import precision_recall_curve, average_precision_score, auc
from sklearn.metrics import roc_auc_score

In [ ]:
import train_model as tm
import visualize as vis
import build_features as bf
import make_dataset as md

In [ ]:
# set random state
rand_state = 1

# Load files into memory

In [ ]:
counts_filename = 'data/count_data_species_raw_WIS_overlapping_fungi_bacteria_12773samples.tsv'
metadata_filename = 'data/metadata_species_WIS_overlapping_fungi_bacteria_12773samples.tsv'

In [ ]:
counts = md.read_fungi_data(counts_filename)
metadata = md.read_fungi_data(metadata_filename)

In [ ]:
# consider selecting metadata columns before combining
combined = pd.merge(metadata, counts, on="sampleid", how="inner")

In [ ]:
metadata.columns

In [ ]:
metadata.head()

In [ ]:
counts.head()

In [ ]:
combined.head()

In [ ]:
metadata.shape

In [ ]:
counts.shape

In [ ]:
combined.shape

In [ ]:
combined["days_to_death"].value_counts()

In [ ]:
combined["days_to_death"].mean()

In [ ]:
# note 76% of samples dont have DTD
combined["days_to_death"].isna().mean()

In [ ]:
# 3036 data points to work with
combined["days_to_death"].notna().sum()

In [ ]:
# drop nans + values < 10k
plt.boxplot(combined[combined['days_to_death'] < 10_000]['days_to_death'].dropna(), vert=False)

## Searching for metadata features

In [ ]:
len(metadata.columns)

In [ ]:
metadata[metadata.columns[1:10]].head()
# important columns
# [experimental_strategy]
# what the hell are these
# [analyte_amount, analyte_A260A280Ratio, aliquot_concentration]

In [ ]:
metadata[metadata.columns[11:20]].head()
# important columns
# [gender, race, ethnicity, disease_type, sample_type]
# what the hell are these
# [data_subtype, cgc_platform]

In [ ]:
metadata[metadata.columns[21:30]].head()
# important columns
# [primary_site, days_to_death]
# what the hell are these
# [reference_genome, investigation]

In [ ]:
metadata[metadata.columns[31:41]].head()
# important columns
# [data_submitting_center_label, tissue_source_site_label, country_of_sample_procurement, pathologic_stage_label]
# what the hell are these
# [portion_is_ffpe, pathologic_t_label, pathologic_n_label, histological_diagnosis_label, PlateCenter, PlateCenterFlag]

## Metadata Regression

In [ ]:
important_cols = ['experimental_strategy', 'gender', 'race', 'ethnicity', 
'disease_type', 'sample_type','primary_site', 'days_to_death','data_submitting_center_label', 
'tissue_source_site_label', 'country_of_sample_procurement', 'pathologic_stage_label']

In [ ]:
metadata['pathologic_stage_label'].value_counts()
# experimental_strategy          - ohe
# gender                         - ohe
# race                           - ohe
# ethnicity                      - ohe
# disease_type                   - ohe
# sample_type                    - ohe (possibly assign numerical? are they ordinal?)
# primary_site                   - ohe
# data_submitting_center_label   - ohe
# tissue_source_site_label       - avoid ohe (179 unique vals)
# country_of_sample_procurement  - ohe
# pathologic_stage_label         - ohe (reduce stages)


# days_to_death             - numerical

## Regression Model

In [ ]:
reg_data = combined[combined["days_to_death"].notna()]

In [ ]:
reg_data.shape

In [ ]:
# separate X and Y and generate 

test_prop = 0.1
reg_X = reg_data.drop(columns=metadata.columns)
reg_Y = reg_data["days_to_death"]
reg_Xtrain, reg_Xtest, reg_Ytrain, reg_Ytest = train_test_split(reg_X, reg_Y, test_size=test_prop, random_state=rand_state)

In [ ]:
reg_model = sk.linear_model.LinearRegression()
reg_model.fit(reg_Xtrain, reg_Ytrain)
preds = reg_model.predict(reg_Xtest)
preds = [x if x > 0 else 0 for x in preds] # Replace negative predictions
scores = mean_squared_error(reg_Ytest, preds)
scores

In [ ]:
(reg_Ytest - preds).head()

## Classification Model

In [ ]:
# taken from train_model and modified for this task

def model_predict(dataset, cancer_stages):
    '''Take in as input the cleaned datasets of the features(X) and the one-hot encoded cancer stages/targets(Y)
       then perform 10-fold validation split and use them to train the model.
       
       Output: Auroc and Aupr scores of the model
       
    '''
    
    # define cross validation hyperparams
    n_splits = 10
    skf_random = 0 #DO NOT TOUCH
    shuffle=True
    # define model hyperparams, can automate tuning later
    loss = 'exponential'
    learning_rate = 0.1
    n_estimators = 150
    max_depth = 3
    clf_random = 0 #DO NOT TOUCH
   
    skf = StratifiedKFold(n_splits=n_splits, random_state=skf_random, shuffle=shuffle)

    clf = GradientBoostingClassifier(loss=loss, learning_rate=learning_rate, n_estimators=n_estimators, max_depth=max_depth, random_state=clf_random)
    
    for i, stage in enumerate(cancer_stages.columns, start=1):
        X = dataset
        y = cancer_stages[stage]
        
        auroc_plt_data = np.array([])
        aupr_plt_data = np.array([])
        
        for train_index, val_index in skf.split(X, y):
            train_X, train_y = X.iloc[train_index], y.iloc[train_index]
            val_X, val_y = X.iloc[val_index], y.iloc[val_index]

            clf.fit(train_X, train_y) #re-fit model
            
            preds = clf.predict_proba(val_X)[:,1] #predict, probability of positive class predict

            auroc = roc_auc_score(val_y, preds) #TODO: implement separate score function to return multiple scores
            aupr = average_precision_score(val_y, preds)
            
            auroc_plt_data = np.append(auroc_plt_data, auroc)
            aupr_plt_data = np.append(aupr_plt_data, aupr)
    
    return auroc_plt_data, aupr_plt_data

In [ ]:
# Week 3 todo
# make wrapper function to process dataset into format to input into model_predict()
    # OHE and combine cancer substages
    # create feature matrix (basically just removing y var)
    # plot auroc and aupr
# cross val for regression?